## Create an HTTPS ingress controller and use your own TLS certificates
Deploy the HAProxy controller in an Azure Kubernetes Service (AKS) cluster. Then generate your own certificates, and create a Kubernetes secret for use with the ingress route. 

## Prerequisites
* Have the latest release of Helm installed.
* Azure CLI version 2.0.64 or later.
* Have HAProxy ingress controller deployed
 
 After the installation of the HAProxy controller, an Azure public IP address is created for the ingress controller. This public IP address is static for the life-span of the ingress controller. If you delete the ingress controller, the public IP address assignment is lost. If you then create an additional ingress controller, a new public IP address is assigned. If you wish to retain the use of the public IP address, you can instead create an ingress controller with a static public IP address.

To get the External IP address under the namespace ingress-controller, use the ```kubectl get service``` command, as follows:

Note:

The namespace ingress-controller was created during the deployment of the HAProxy ingress controller.

In [ ]:
$ kubectl get svc -n ingress-controller

NAME              TYPE           CLUSTER-IP     EXTERNAL-IP    PORT(S)                      AGE
haproxy-ingress   LoadBalancer   10.0.241.105   20.85.19.234   80:31252/TCP,443:31722/TCP   24s

It takes a few minutes for the IP address to be assigned to the service.  Make a note of this public IP address, as it's used in the last step to test the deployment.  Since no ingress rules have been created yet, If you browse to the public IP address, the ingress controller's default 404 page is displayed.

## Generate TLS certificates
For this article, a self-signed certificate is generated with openssl. For production use, you should request a trusted, signed certificate through a provider or your own certificate authority (CA).

The following example generates a 2048-bit RSA X509 certificate valid for 365 days named ecl-tls.crt. The private key file is named ecl-tls.key. A Kubernetes TLS secret requires both of these files.

This article works with the demo.azure.com subject common name and doesn't need to be changed. For production use, specify your own organizational values for the -subj parameter:

In [ ]:
openssl req -x509 -nodes -days 365 -newkey rsa:2048 \
    -out ecl-tls.crt \
    -keyout ecl-tls.key \
    -subj "/CN=eclwatch.com/O=eclwatch-tls"

You should see something like this:

In [ ]:
Generating a RSA private key
.+++++
...........................................................+++++
writing new private key to 'ecl-tls.key'
-----

## Create Kubernetes secret for the TLS certificate
To allow Kubernetes to use the TLS certificate and private key for the ingress controller, you create and use a Secret. The secret is defined once, and uses the certificate and key file created in the previous step. You then reference this secret when you define ingress routes.

The following example creates a Secret name ecl-tls:

In [ ]:
$ kubectl create secret tls ecl-tls \
>     --key ecl-tls.key \
>     --cert ecl-tls.crt

# Output:
secret/ecl-tls created

## Create an Ingress route
The ingress resource configures the rules that route traffic to the backend server eclwatch through port number 8010.

If the host name specified during the certificate request process, the CN name, doesn't match the host defined in your ingress route, you ingress controller displays a Kubernetes Ingress Controller Fake Certificate warning. The CN name in this example is 'eclwatch.com'.

To create the ingress resource, specify the CN name as host, and tls secret name.  Open a file, such as eclwatch-ingress.yaml, and copy in the following:

apiVersion: networking.k8s.io/v1beta1
kind: Ingress
metadata:
  name: eclwatch-ingress
  annotations:
    kubernetes.io/ingress.class: haproxy
spec:
  tls:
  - hosts:
    - eclwatch.com
    secretName: ecl-tls
  rules:
  - host: eclwatch.com
    http:
      paths:
      - backend:
          serviceName: eclwatch
          servicePort: 8010
        path: /

Now apply the ingress route using the following command:

In [ ]:
$ kubectl apply -f eclwatch-ingress.yaml

ingress.networking.k8s.io/eclwatch-ingress created

## Test the ingress configuration
To test the certificates with the fake eclwatch.com host, use curl and specify the --resolve parameter. This parameter lets you map the eclwatch.com name to the public IP address of your ingress controller. Specify the public IP address of your own ingress controller, as shown in the following example:

In [ ]:
curl -v -k --resolve eclwatch.com:443:20.85.19.234 https://eclwatch.com

* Connected to 20.72.124.58 (20.72.124.58) port 443 (#0)
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: /Users/amyma/opt/anaconda3/ssl/cacert.pem
  CApath: none
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
* TLSv1.3 (IN), TLS handshake, Server hello (2):
* TLSv1.3 (IN), TLS handshake, Encrypted Extensions (8):
* TLSv1.3 (IN), TLS handshake, Certificate (11):
* TLSv1.3 (IN), TLS handshake, CERT verify (15):
* TLSv1.3 (IN), TLS handshake, Finished (20):
* TLSv1.3 (OUT), TLS change cipher, Change cipher spec (1):
* TLSv1.3 (OUT), TLS handshake, Finished (20):
* SSL connection using TLSv1.3 / TLS_AES_128_GCM_SHA256
* ALPN, server accepted to use http/1.1
* Server certificate:
*  subject: O=Acme Co; CN=Kubernetes Ingress Controller Fake Certificate
*  start date: Jul 22 03:37:00 2021 GMT
*  expire date: Jul 22 03:37:00 2022 GMT
*  issuer: O=Acme Co; CN=Kubernetes Ingress Controller Fake Certificate
*  SSL certificate verify result: self signed certificate (18), continuing anyway.

The -v parameter in the curl command outputs verbose information, including the TLS certificate received. Half-way through your curl output, you can verify that your own TLS certificate was used. The -k parameter continues loading the page even though a self-signed certificate is being used.

## Clean up resources

1. Delete the sample namespace and all resources

To delete the entire namespace, use the ```kubectl delete``` command and specify the namespace name of the HAProxy controller, 'ingress-controller'. All the resources in the namespace are deleted.

In [ ]:
kubectl delete namespace ingress-controller

2. Remove the ingress route that directed traffic to the sample apps:

In [ ]:
kubectl delete -f eclwatch-ingress.yaml

3. Delete the certificate Secret:

In [ ]:
kubectl delete secret ecl-tls